# Giving a Chat Model Memory



In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import os

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
chat_llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

#### Add History to the Prompt

As each call to the LLM is stateless, you need to include the chat history in every call to the LLM. You can modify the prompt template to include the chat history as a list of messages using a MessagesPlaceholder object.

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

#### Chat Message History

To keep the message history, you will need to wrap the chat_chain in a Runnable. Specifically, you will use the RunnableWithMessageHistory runnable which will use the memory component to store and retrieve the conversation history.

First, you will need to create a ChatMessageHistory memory component and a function that the RunnableWithMessageHistory will use to get the memory component.

In [6]:
memory = ChatMessageHistory()

def get_memory(session_id):
    return memory

The `get_memory()` function will return the ChatMessageHistory memory component. Note how it expects a session_id parameter, this would be used to identify the specific conversation (or session). As there will only be one conversation in memory at a time, you can ignore this parameter.

You can now create a new chain using the RunnableWithMessageHistory, passing the chat_chain and the get_memory function.

In [7]:
chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)

The input_messages_key and history_messages_key parameters are the keys in the prompt that will be populated with the user’s question and the chat history.

#### Invoke the Chat Model

When you call the `chat_with_message_history` chain, the user’s question and the response will be stored in the ChatMessageHistory memory component.

Every subsequent call to the `chat_with_message_history` chain will include the chat history in the prompt.

When you ask the chat model multiple questions, the LLM will use the context from the previous questions when responding.

In [8]:
current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while True:
    question = input("> ")

    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": "none"}
        }
    )
    
    print(response)

Dude, at Watergate Bay, we got some sweet 3ft waves but the winds are a bummer, they're totally onshore. It's still worth catching some waves though!
Dude, you're at Watergate Bay. The waves are 3ft and the winds are onshore. Time to shred some gnarly waves, bro!


KeyboardInterrupt: Interrupted by user